In [ ]:
from nevernegative import SimpleScanner, SimpleScannerConfig
from nevernegative.crop import HoughCropperConfig
from nevernegative.color import HistogramScalingConfig
# or
from nevernegative.config import HoughCropperConfig, HistogramScalingConfig
from nevernegative.layers import Resize, Blur, Threshold, EdgeDetect, HoughCrop
from nevernegative.callbacks import PlottingCallback
import numpy as np

## Configuration Concepts

### Version 1

In [ ]:
config = SimpleScannerConfig(
    cropping=HoughCropperConfig(
        min_distance=30,
        peak_threshold_ratio=0.2,
        thresholding={"type": "mean"}, # required
        edge_detection={"type": "canny"}, # required
        transformations=[ # optional
            Resize(ratio=0.5),
            Blur(),
        ],
        # callbacks=[PlottingCallback()],
    )
    color=HistogramScalingConfig(
        ...
        callbacks=[],
    ),
    # or
    color=None,
    pool_size=1,
    callbacks=[PlottingCallbackConfig()]
)

### Version 2: Dependency Tree

In [ ]:
config = SimpleScannerConfig(
    transformations={
        "resized": Resize("original", ratio=0.5),
        "blurred": Blur("resized", (9, 9)),
        "cropped": HoughCrop(
            "blurred",
            transformed_image="original", # or None
            min_distance=30,
            peak_threshold_ratio=0.2,
            thresholding={"type": "mean"}, # required
            edge_detection={"type": "canny"}, # required
            # callbacks=[PlottingCallback()],
        ),
        "balanced": HistogramScalingConfig(
            source="cropped"
            # callbacks=[],
        )
    },
    pool_size=10,
    callbacks=[PlottingCallback()]
)

### Version 3: Layered Functions

In [ ]:
scanner = SimpleScannerConfig(
    transformations=[
        CheckerboardUnwarp(),
        HoughUnwarp(),
        Resize(name="resized", ratio=0.5),
        Blur((9, 9)),
        HoughCrop(
            name="cropped",
            apply_to="original", # or None
            min_distance=30,
            peak_threshold_ratio=0.2,
            thresholder=Threshold(type="mean"),
            edge_detector=EdgeDetect(type="canny"),
            callbacks=[PlottingCallback()],
        ),
        HistogramScalingConfig()
    ],
    pool_size=10,
    cache_intermediate=False,
    callbacks=[PlottingCallback()]
)

In [ ]:
config = SimpleScannerConfig(
    dewarping=HoughBarrelDewarp( # CheckboardDewarp
        num_points=10,
        batch_average=False,
        center="center" # or [1000, 2000]
    ),
    cropping=HoughCrop(
        min_distance=30,
        hough_peak_ratio=0.2,
        thresholder=Threshold(type="mean"),
        edge_detector=EdgeDetect(type="canny"),
        transformations=[
            Resize(ratio=0.5),
            Blur((9, 9)),
        ],
        callbacks=[PlottingCallback()],
    ),
    color_balancing=HistogramScaler(
        invert=True,
        padding=0.1,
    ),
    pool_size=10,
    cache_intermediate=False,
    callbacks=[PlottingCallback()]
)

## Runtime Flow

In [ ]:
scanner = SimpleScanner(config)

result = scanner.array(np.array([[1, 2, 3]]))
result = scanner.file("path/to/file.cr2", "processed/file.png", raw=True)

scanner.glob("folder/*", "processed/")